In [1]:
"""
Using to artificially insert some 3 labels into some brats22 samples to see the warning
we are getting: 
nnUNetTrainer.py:712: RuntimeWarning: invalid value encountered in scalar divide
  global_dc_per_class = [i for i in [2 * i / (2 * i + j + k) for i, j, k in

hopefully go away
"""

'\nUsing to artificially insert some 3 labels into some brats22 samples to see the warning\nwe are getting: \nnnUNetTrainer.py:712: RuntimeWarning: invalid value encountered in scalar divide\n  global_dc_per_class = [i for i in [2 * i / (2 * i + j + k) for i, j, k in\n\nhopefully go away\n'

In [2]:
import os
import sys
import numpy as np

import nibabel as nib

In [3]:
sys.path.append('/home/edwardsb/repositories/private-mri-sandbox/2D_data_creation')

from nifti_utils import load_nifti, save_nifti

In [4]:
datadir = '/raid/edwardsb/projects/RANO/ExampleBraTS22_Labels0-4'
# this next one is for when we replace labels with those that have 3s inserted into it
new_datadir = '/raid/edwardsb/projects/RANO/FAKE_withlabel3_ExampleBraTS22_Labels0-4'

In [5]:
def profile_labels(label_path):
    array = load_nifti(label_path)
    print(f"The labels represented in the file: {label_path.split('/')[-1]} are: {np.unique(array)} and array has shape: {array.shape}\n")

In [6]:
def get_modality(niftidir, ending):
    # I check that the ending occurs exactly once
    fnames = os.listdir(niftidir)
    ours = [fname for fname in fnames if fname.endswith(ending)]
    if len(ours) ==0:
        raise ValueError(f"The ending {ending} was not observed in the nifti pardir: {niftidir}")
    elif len(ours) > 1:
        raise ValueError(f"The ending {ending} occurs {len(ours)} times when only once was expected.")
    else:
        return os.path.join(niftidir, ours[0])



In [7]:


def modalitypaths(nifti_pardir, ending, new_nifti_pardir=None, create_new=False):
    modalitypaths = []
    subnames = os.listdir(nifti_pardir)
    subpaths = [os.path.join(nifti_pardir, name) for name in subnames]
    subdirpaths = [path for path in subpaths if os.path.isdir(path)]

    for path in subdirpaths:
        modalitypaths.append(get_modality(niftidir=path, ending=ending))
    print(f"Found {len(modalitypaths)} paths with modality {ending} under parent directory: {nifti_pardir}")

    if not create_new:
        return modalitypaths
    else:
        # creating paths for similar files in alternate folder (note the modality can already exist here so be careful as overwriting will happen)
        new_modalitypaths = [os.path.join(new_nifti_pardir, path.split('/')[-2], path.split('/')[-1]) for path in modalitypaths]

        return modalitypaths, new_modalitypaths


In [8]:
def profile_all_masks(nifti_pardir):
    paths = modalitypaths(nifti_pardir=nifti_pardir, ending='_seg.nii.gz')
    for path in paths:
        profile_labels(label_path=path)

In [9]:
profile_all_masks(nifti_pardir=datadir)

Found 8 paths with modality _seg.nii.gz under parent directory: /raid/edwardsb/projects/RANO/ExampleBraTS22_Labels0-4
The labels represented in the file: BraTS2021_00344_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00343_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00349_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00347_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00340_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00348_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00341_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2

In [10]:
def insert_label3(labelpath, new_labelpath, exists_ok=False):                  
    array = load_nifti(labelpath)
    array[120:124, 120:124, 73:75] = 3
    print(f"Brandon DEBUG - type of thing I think is old label image is: {type(nib.load(labelpath))}")
    save_nifti(array=array, path=new_labelpath, exists_ok=exists_ok, metadata_from=nib.load(labelpath))

In [11]:
def create_new_masks(nifti_pardir, fakesegs_nifti_pardir, new_path_exists_ok=False):
    paths, new_paths = modalitypaths(nifti_pardir=nifti_pardir, ending='_seg.nii.gz', new_nifti_pardir=fakesegs_nifti_pardir, create_new=True)
    for path, new_path in zip(paths, new_paths):
        profile_labels(label_path=path)
        insert_label3(labelpath=path, new_labelpath=new_path, exists_ok=new_path_exists_ok)
        profile_labels(label_path=new_path)

In [12]:
create_new_masks(nifti_pardir=datadir, fakesegs_nifti_pardir=new_datadir, new_path_exists_ok=True)

Found 8 paths with modality _seg.nii.gz under parent directory: /raid/edwardsb/projects/RANO/ExampleBraTS22_Labels0-4
The labels represented in the file: BraTS2021_00344_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

Brandon DEBUG - type of thing I think is old label image is: <class 'nibabel.nifti1.Nifti1Image'>
The labels represented in the file: BraTS2021_00344_seg.nii.gz are: [0. 1. 2. 3. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00343_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

Brandon DEBUG - type of thing I think is old label image is: <class 'nibabel.nifti1.Nifti1Image'>
The labels represented in the file: BraTS2021_00343_seg.nii.gz are: [0. 1. 2. 3. 4.] and array has shape: (240, 240, 155)

The labels represented in the file: BraTS2021_00349_seg.nii.gz are: [0. 1. 2. 4.] and array has shape: (240, 240, 155)

Brandon DEBUG - type of thing I think is old label image is: <class 'nibabel.nif